In [34]:
import os 
import numpy as np
import json
from math import sqrt
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt

In [39]:
jsonexample = json.load(open("c/WIN_20230130_08_31_47_Pro.json"))
shape = jsonexample["shapes"]
flatlist = []
for points in shape:
    for point in points["points"]:
        flatlist.append(point)
flatlist = np.array(flatlist)

def min_bounding_rect(points):
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.
    hull_points = points[ConvexHull(points).vertices]
    
    edges = np.zeros((len(hull_points)-1,2))
    edges = hull_points[1:] - hull_points[:-1]
    
    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])
    
    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)
    
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles - pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1,2,2))
    
    rot_points = np.dot(rotations, hull_points.T)
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)
    
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)
    
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    print(x1,y1,x2,y2)
#     r = rotations[best_idx]
    
#     rval = np.zeros((4,2))
#     rval[0] = np.dot([x1,y2], r)
#     rval[1] = np.dot([x2, y2], r)
#     rval[2] = np.dot([x2, y1], r)
#     rval[3] = np.dot([x1, y1], r)
#     print(rval)
#     return rval

rect = min_bounding_rect(flatlist)

1616.3700222939854 903.6687585747296 403.660398539549 131.58254537429625


/tmp/ipykernel_29508/964221716.py:10: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate


In [38]:
rect.append(rect[0])
xs , ys = zip(*rect)
plt.figure()
plt.plot(xs,ys)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'append'